# Working with `DeepSeek-R1-Distill-Qwen-1.5B`

- Distilled version of DeepSeek R1, with 1.5B params rather than the original 685B params!
- Based on the Qwen 2.5 model

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import evaluate
import numpy as np

# Importing the model
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

/Users/edrikchong/VSCode-Projects/intro-LLM-implementation/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---
# Fine-tuning the model

### Loading the dataset

In this case, we are using the *Great Gatsby* to train the model.

In [2]:
from datasets import load_dataset

# Loading dataset from hugging face (Great Gatsby txt)
ds = load_dataset("TeacherPuffy/book")

# This line prints out the "train" split where each index is a line number
print(ds["train"][100])


{'text': 'Sometimes she and Miss Baker talked at the same time, unobtrusively and with a playful banter that was never quite chatter, as cool as their white dresses and their impersonal eyes, devoid of all desire. They were here—and they accepted Tom and me, making only a polite effort to entertain or be entertained. They knew that dinner would soon be over, and a little later, the evening too would end and be casually put away. It was a stark contrast to the West, where an evening was rushed from one phase to the next, driven by a continually disappointed anticipation or sheer nervous dread of the moment itself.'}


### Tokenization
We are using a tokenizer to process the text and provide padding as well as a truncation strategy to handle varying sequence lengths. The `map` method is used to apply the preprocessing function over the entire dataset

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id  # Ensure pad_token_id is set

def tokenize_function(examples):
    outputs = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
    outputs["labels"] = outputs["input_ids"].copy()  # Set labels to be identical to input_ids
    return outputs

tokenized_datasets = ds.map(tokenize_function)
print(tokenized_datasets)



DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 111
    })
})


Then, to prepare for training, remove and edit columns that hugging face expects.
Here, the text column is removed, keeping `input_ids`, `attention_mask`, and `labels`

In [4]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.with_format("torch")
print(tokenized_datasets["train"])


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 111
})


---
# Training the model with PyTorch Trainer

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto")
model.resize_token_embeddings(len(tokenizer))

# Contains all hyperparameters
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch", num_train_epochs=1)

# Computes and reports metrics during training
metric = evaluate.load("accuracy")

# Calculates accuracy of the predictions
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset='no',
    compute_metrics=compute_metrics,
    
)


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [6]:
trainer.train()

RuntimeError: MPS backend out of memory (MPS allocated: 8.32 GB, other allocations: 596.69 MB, max allowed: 9.07 GB). Tried to allocate 592.44 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).